In [21]:
import tensorflow as tf
import numpy as np

In [22]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [4]:
#input

In [5]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,28,28,1])

In [6]:
#set up the graph

In [7]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [8]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [9]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [10]:
#Dropout

In [11]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [12]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [13]:
#Training by build graph

In [14]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [15]:
#Traing the model by session

In [16]:
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.82
step 200, training accuracy 0.9
step 300, training accuracy 0.86
step 400, training accuracy 0.98
step 500, training accuracy 0.96
step 600, training accuracy 1
step 700, training accuracy 0.94
step 800, training accuracy 0.9
step 900, training accuracy 1
step 1000, training accuracy 0.98
step 1100, training accuracy 0.96
step 1200, training accuracy 0.96
step 1300, training accuracy 0.96
step 1400, training accuracy 0.96
step 1500, training accuracy 1
step 1600, training accuracy 0.98
step 1700, training accuracy 0.96
step 1800, training accuracy 0.92
step 1900, training accuracy 0.98
step 2000, training accuracy 1
step 2100, training accuracy 0.98
step 2200, training accuracy 1
step 2300, training accuracy 0.98
step 2400, training accuracy 0.98
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, trainin

In [17]:
#Dropout is often very effective at reducing overfitting, 
#but it is most useful when training very large neural networks. 

In [23]:
from keras.layers import merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import Input
from keras.preprocessing.image import load_img, img_to_array
import keras.backend as K
import numpy as np

In [24]:

def identity_block(input_tensor, kernel_size, filters, stage, block):
    """
    the identity_block is the block that has no conv layer at shortcut
    params:
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    """
    dim_ordering = K.image_dim_ordering()
    nb_filter1, nb_filter2, nb_filter3 = filters
    if dim_ordering == 'tf':
        axis = 3
    else:
        axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    out = Convolution2D(nb_filter1, 1, 1, dim_ordering=dim_ordering, name=conv_name_base + '2a')(input_tensor)
    out = BatchNormalization(axis=axis, name=bn_name_base + '2a')(out)
    out = Activation('relu')(out)

    out = out = Convolution2D(nb_filter2, kernel_size, kernel_size, border_mode='same',
                              dim_ordering=dim_ordering, name=conv_name_base + '2b')(out)
    out = BatchNormalization(axis=axis, name=bn_name_base + '2b')(out)
    out = Activation('relu')(out)

    out = Convolution2D(nb_filter3, 1, 1, dim_ordering=dim_ordering, name=conv_name_base + '2c')(out)
    out = BatchNormalization(axis=axis, name=bn_name_base + '2c')(out)

    out = merge([out, input_tensor], mode='sum')
    out = Activation('relu')(out)
    return out


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """
    conv_block is the block that has a conv layer at shortcut
    params:
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the nb_filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    Note that from stage 3, the first conv layer at main path is with subsample=(2,2)
    And the shortcut should has subsample=(2,2) as well
    """
    nb_filter1, nb_filter2, nb_filter3 = filters
    dim_ordering = K.image_dim_ordering()
    if dim_ordering == 'tf':
        axis = 3
    else:
        axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    out = Convolution2D(nb_filter1, 1, 1, subsample=strides,
                        dim_ordering=dim_ordering, name=conv_name_base + '2a')(input_tensor)
    out = BatchNormalization(axis=axis, name=bn_name_base + '2a')(out)
    out = Activation('relu')(out)

    out = Convolution2D(nb_filter2, kernel_size, kernel_size, border_mode='same',
                        dim_ordering=dim_ordering, name=conv_name_base + '2b')(out)
    out = BatchNormalization(axis=axis, name=bn_name_base + '2b')(out)
    out = Activation('relu')(out)

    out = Convolution2D(nb_filter3, 1, 1, dim_ordering=dim_ordering, name=conv_name_base + '2c')(out)
    out = BatchNormalization(axis=axis, name=bn_name_base + '2c')(out)

    shortcut = Convolution2D(nb_filter3, 1, 1, subsample=strides,
                             dim_ordering=dim_ordering, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=axis, name=bn_name_base + '1')(shortcut)

    out = merge([out, shortcut], mode='sum')
    out = Activation('relu')(out)
    return out



In [25]:
K.image_dim_ordering()

'tf'

In [32]:
import tensorflow as tf

tf.python.control_flow_ops = tf

inputs = Input(shape=(28,28,1))
    
    
dim_ordering = K.image_dim_ordering()
out = ZeroPadding2D((3, 3), dim_ordering=dim_ordering)(inputs)
out = Convolution2D(64, 3, 3, subsample=(1, 1), dim_ordering=dim_ordering, name='conv1')(out)
out = Activation('relu')(out)

out = conv_block(out, 3, [32, 32, 128], stage=2, block='a', strides=(1, 1))
out = identity_block(out, 3, [32, 32, 128], stage=2, block='b')
out = identity_block(out, 3, [32, 32, 128], stage=2, block='c')
out = AveragePooling2D((7, 7), dim_ordering=dim_ordering)(out)

out = Flatten()(out)
out = Dense(10, activation='softmax', name='fc1000')(out)
#out = Convolution2D(64, 7, 7, subsample=(2, 2), dim_ordering=dim_ordering, name='conv1')(out)
#out = BatchNormalization(axis=axis, name='bn_conv1')(out)
#out = Activation('relu')(out)
#out = MaxPooling2D((3, 3), strides=(2, 2), dim_ordering=dim_ordering)(out)

'''
out = conv_block(out, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
out = identity_block(out, 3, [64, 64, 256], stage=2, block='b')
out = identity_block(out, 3, [64, 64, 256], stage=2, block='c')

out = conv_block(out, 3, [128, 128, 512], stage=3, block='a')
out = identity_block(out, 3, [128, 128, 512], stage=3, block='b')
out = identity_block(out, 3, [128, 128, 512], stage=3, block='c')
out = identity_block(out, 3, [128, 128, 512], stage=3, block='d')

out = conv_block(out, 3, [256, 256, 1024], stage=4, block='a')
out = identity_block(out, 3, [256, 256, 1024], stage=4, block='b')
out = identity_block(out, 3, [256, 256, 1024], stage=4, block='c')
out = identity_block(out, 3, [256, 256, 1024], stage=4, block='d')
out = identity_block(out, 3, [256, 256, 1024], stage=4, block='e')
out = identity_block(out, 3, [256, 256, 1024], stage=4, block='f')

out = conv_block(out, 3, [512, 512, 2048], stage=5, block='a')
out = identity_block(out, 3, [512, 512, 2048], stage=5, block='b')
out = identity_block(out, 3, [512, 512, 2048], stage=5, block='c')

out = AveragePooling2D((7, 7), dim_ordering=dim_ordering)(out)
out = Flatten()(out)
out = Dense(10, activation='softmax', name='fc1000')(out)
'''
model = Model(inputs, out)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
batch = mnist.train.next_batch(10000)
#model.fit(data, labels)  # starts training
temp = np.reshape(batch[0], (10000,28,28,1))
model.fit(temp, batch[1],validation_split=0.2,nb_epoch=15)  # starts training
#model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10)

#You can do this by setting the validation_split argument on the fit() function to a percentage of the size of your training dataset.


Train on 8000 samples, validate on 2000 samples
Epoch 1/15
8000/8000 [==============================] - 297s - loss: 0.6052 - acc: 0.8527 - val_loss: 2.4655 - val_acc: 0.1055
Epoch 2/15
8000/8000 [==============================] - 296s - loss: 0.1497 - acc: 0.9543 - val_loss: 1.6169 - val_acc: 0.4695
Epoch 3/15
8000/8000 [==============================] - 288s - loss: 0.1090 - acc: 0.9674 - val_loss: 0.1767 - val_acc: 0.9450
Epoch 4/15
8000/8000 [==============================] - 288s - loss: 0.0897 - acc: 0.9738 - val_loss: 0.1383 - val_acc: 0.9685
Epoch 5/15
8000/8000 [==============================] - 293s - loss: 0.0788 - acc: 0.9770 - val_loss: 0.1072 - val_acc: 0.9665
Epoch 6/15
8000/8000 [==============================] - 290s - loss: 0.0645 - acc: 0.9786 - val_loss: 0.1196 - val_acc: 0.9655
Epoch 7/15
8000/8000 [==============================] - 306s - loss: 0.0597 - acc: 0.9819 - val_loss: 0.0828 - val_acc: 0.9765
Epoch 8/15
8000/8000 [==============================] - 309s - 

In [31]:
from keras.models import Sequential
import pandas as pd

X_train,y_train=mnist.train.next_batch(10000)
X_test,y_test=mnist.test.next_batch(10000)
X_train = X_train.reshape(X_train.shape[0],  28, 28,1).astype('float32') 
X_test = X_test.reshape(X_test.shape[0], 28, 28,1).astype('float32') 
Y_train = y_train
Y_test = y_test




print("Test classification rate %0.05f" % model.evaluate(X_test, Y_test)[1])
#y_hat = model.predict(X_test)
#print(y_hat)
#pd.crosstab(y_hat, y_test[0])
#y_test.shape

10000/10000 [==============================] - 116s   
Test classification rate 0.97670


KeyboardInterrupt: 

In [29]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 28, 28, 1)     0                                            
____________________________________________________________________________________________________
zeropadding2d_8 (ZeroPadding2D)  (None, 34, 34, 1)     0           input_8[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 32, 32, 64)    640         zeropadding2d_8[0][0]            
____________________________________________________________________________________________________
activation_10 (Activation)       (None, 32, 32, 64)    0           conv1[0][0]                      
___________________________________________________________________________________________

In [53]:
mnist.test.images.shape

(10000, 784)

In [13]:
from keras.layers import Input, Dense
from keras.models import Model

# this returns a tensor
inputs = Input(shape=(784,))
#keras.layers.core.Dense(output_dim, init='glorot_uniform', activation='linear',
#                        weights=None, W_regularizer=None, b_regularizer=None, 
#                        activity_regularizer=None, W_constraint=None, b_constraint=None,
#                        bias=True, input_dim=None)
# a layer instance is callable on a tensor, and returns a tensor
#                      
Just your regular fully connected NN layer.
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# this creates a model that includes
# the Input layer and three Dense layers
model = Model(input=inputs, output=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
batch = mnist.train.next_batch(50)
#model.fit(data, labels)  # starts training
model.fit(batch[0], batch[1])  # starts training

Epoch 1/10
50/50 [==============================] - 0s - loss: 2.3782 - acc: 0.0800     
Epoch 2/10
50/50 [==============================] - 0s - loss: 1.9542 - acc: 0.5400     
Epoch 3/10
50/50 [==============================] - 0s - loss: 1.6766 - acc: 0.6000     
Epoch 4/10
50/50 [==============================] - 0s - loss: 1.4642 - acc: 0.7400     
Epoch 5/10
50/50 [==============================] - 0s - loss: 1.2798 - acc: 0.7800     
Epoch 6/10
50/50 [==============================] - 0s - loss: 1.1167 - acc: 0.8000     
Epoch 7/10
50/50 [==============================] - 0s - loss: 0.9846 - acc: 0.8200     
Epoch 8/10
50/50 [==============================] - 0s - loss: 0.8629 - acc: 0.9000     
Epoch 9/10
50/50 [==============================] - 0s - loss: 0.7664 - acc: 0.9200     
Epoch 10/10
50/50 [==============================] - 0s - loss: 0.6709 - acc: 0.9600     


In [67]:
%pylab inline

import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from keras.datasets import mnist, cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras.regularizers import l2
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [73]:
X_train,y_train=mnist.train.next_batch(10000)
X_test,y_test=mnist.test.next_batch(10000)
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32') 
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32') 
Y_train = y_train
Y_test = y_test

In [74]:

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape = (1, 28, 28)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
          
rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms)

In [75]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10,
          verbose=1, show_accuracy=True, validation_split=0.1)

/home/king/anaconda2/lib/python2.7/site-packages/keras/models.py:603: UserWarning: The "show_accuracy" argument is deprecated, instead you should pass the "accuracy" metric to the model at compile time:
`model.compile(optimizer, loss, metrics=["accuracy"])`
  warnings.warn('The "show_accuracy" argument is deprecated, '


Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 6s - loss: 2.2528 - val_loss: 2.1561
Epoch 2/10
9000/9000 [==============================] - 6s - loss: 1.9858 - val_loss: 1.8111
Epoch 3/10
9000/9000 [==============================] - 5s - loss: 1.5657 - val_loss: 1.3738
Epoch 4/10
4448/9000 [=============>................] - ETA: 3s - loss: 1.2404

ValueError: I/O operation on closed file